# movielens k-fold dataset generation

In [ ]:
import os
os.chdir('./')
import numpy as np
import random as random
import pandas as pd
import scipy.sparse as sp
import random

import torch.utils.data as data

In [ ]:
raw = pd.read_csv('./rawdata/ratings.dat', sep = "::", header = None,engine = 'python')
raw.columns = ['uid', 'sid', 'ratings', 'timestamp']
print(raw)
raw = raw[['uid', 'sid']]
print(raw)

In [ ]:
new_items = raw.sid.value_counts()[raw.sid.value_counts() >= 10].reset_index()['index'].values

In [ ]:
data2 = raw[raw.sid.isin(new_items)]
data2 = data2.reset_index()[['uid', 'sid']]
print(data2)

### Reindex by popularity count

In [ ]:
pop_uid = data2.uid.value_counts().reset_index()
pop_uid.columns = ['uid', 'uid_counts']
print(pop_uid)

In [ ]:
pop_uid_dict = pop_uid.reset_index()
pop_uid_dict = pop_uid_dict[['index', 'uid']]
pop_uid_dict.columns = ['new_uid', 'uid']
print(pop_uid_dict)
pop_uid_dict = dict(zip(pop_uid_dict.uid, pop_uid_dict.new_uid))
print(pop_uid_dict)

In [ ]:
pop_sid = data2.sid.value_counts().reset_index()
pop_sid.columns = ['sid', 'sid_counts']
print(pop_sid)

In [ ]:
pop_sid_dict = pop_sid.reset_index()
pop_sid_dict = pop_sid_dict[['index', 'sid']]
pop_sid_dict.columns = ['new_sid', 'sid']
print(pop_sid_dict)
pop_sid_dict = dict(zip(pop_sid_dict.sid, pop_sid_dict.new_sid))
print(pop_sid_dict)

In [ ]:
print(data2)
data2['uid'] = data2.uid.map(pop_uid_dict).values
data2['sid'] = data2.sid.map(pop_sid_dict).values
print(data2)

In [ ]:
data2 = data2.sort_values(['uid', 'sid'], ascending = [True, True])
print(data2)

tmp = data2
tmp['one'] = 1

sns.heatmap(tmp.pivot_table(index = 'uid', columns = 'sid', values = 'one', fill_value = 0))

In [ ]:
data2 = data2.reset_index()[['uid', 'sid']]
print(data2)

In [ ]:
all_data = data2

In [ ]:
all_data = all_data.reset_index()[['uid', 'sid']]
print(all_data)

### Folds Split

#### Número de K_Folds

In [ ]:
# Número de Folds a serem gerados
num_folds = 5

In [ ]:
# random.seed(0)

fold_1 = all_data.sample(frac = 1/num_folds)
fold_1 = fold_1.reset_index()
fold_1 = fold_1.drop(columns = ['index'])

rest = all_data.merge(fold_1.drop_duplicates(), on=['uid','sid'], how='left', indicator=True)

rest = rest[rest['_merge'] == 'left_only']
rest = rest.drop(columns = ['_merge'])

name = 1

for i in range(num_folds - 2):

    name += 1
    num_folds -= 1

    globals()['fold_'+str(name)] = rest.sample(frac = 1/num_folds)
    globals()['fold_'+str(name)] = globals()['fold_'+str(name)].reset_index()
    globals()['fold_'+str(name)] = globals()['fold_'+str(name)].drop(columns = ['index'])

    rest = rest.merge(globals()['fold_'+str(name)].drop_duplicates(), on=['uid','sid'], how='left', indicator=True)

    rest = rest[rest['_merge'] == 'left_only']
    rest = rest.drop(columns = ['_merge'])

name += 1
num_folds -= 1

globals()['fold_'+str(name)] = rest.sample(frac = 1/num_folds)
globals()['fold_'+str(name)] = globals()['fold_'+str(name)].reset_index()
globals()['fold_'+str(name)] = globals()['fold_'+str(name)].drop(columns = ['index'])

# for i in range(name): print('fold_'+str(i +1) + ' =\n',globals()['fold_'+str(i + 1)])

### Make negative sample

In [ ]:
# random.seed(0)

n_user = len(all_data.uid.unique())
n_item = len(all_data.sid.unique())

item_set = set(list(range(n_item)))
neg_sample_df = pd.DataFrame({'uid' : [], 'sid' : []})

for user in list(range(n_user)):
    true_set = all_data[all_data['uid'] == user]['sid'].values
    true_set = set(true_set)
    user_neg_samples = item_set - true_set
    user_neg_samples = list(user_neg_samples)

    list_len = len(user_neg_samples)
    user_neg_samples = random.sample(user_neg_samples, 100)
    tmp_neg_sample_df = pd.DataFrame({'uid' : [user]*100, 'sid' : user_neg_samples})

    neg_sample_df = pd.concat([neg_sample_df, tmp_neg_sample_df])
neg_sample_df['uid'] = neg_sample_df['uid'].astype(int)
neg_sample_df['sid'] = neg_sample_df['sid'].astype(int)
neg_sample_df['type'] = 'neg'
neg_sample_df = neg_sample_df.reset_index()[['uid', 'sid', 'type']]
# print(neg_sample_df)

### Training Val Data Split

In [ ]:
for i in range(name):
    train_part = all_data.merge(globals()['fold_'+str(i + 1)].drop_duplicates(), on=['uid','sid'], how='left', indicator=True)
    train_part = train_part[train_part['_merge'] == 'left_only']
    train_part = train_part.drop(columns = ['_merge'])
    globals()['train_fold_'+str(i + 1)] = train_part.sample(frac = 0.75)
    globals()['train_fold_'+str(i + 1)] = globals()['train_fold_'+str(i + 1)].reset_index()
    globals()['train_fold_'+str(i + 1)] = globals()['train_fold_'+str(i + 1)].drop(columns = ['index'])
    # print('train_fold_'+str(i + 1) + ' =\n',globals()['train_fold_'+str(i + 1)])
    globals()['val_fold_'+str(i + 1)] = train_part.merge(globals()['train_fold_'+str(i + 1)].drop_duplicates(), on=['uid','sid'], how='left', indicator=True)
    globals()['val_fold_'+str(i + 1)] = globals()['val_fold_'+str(i + 1)][globals()['val_fold_'+str(i + 1)]['_merge'] == 'left_only']
    globals()['val_fold_'+str(i + 1)] = globals()['val_fold_'+str(i + 1)].reset_index()
    globals()['val_fold_'+str(i + 1)] = globals()['val_fold_'+str(i + 1)].drop(columns = ['index','_merge'])
    globals()['val_fold_'+str(i + 1)]['type'] = 'pos'
    # print('val_fold_'+str(i + 1) + ' =\n',globals()['val_fold_'+str(i + 1)])
    globals()['val_fold_'+str(i + 1) + '_with_neg'] = pd.concat([globals()['val_fold_'+str(i + 1)], neg_sample_df])
    globals()['val_fold_'+str(i + 1) + '_with_neg'] = globals()['val_fold_'+str(i + 1) + '_with_neg'].reset_index()
    globals()['val_fold_'+str(i + 1) + '_with_neg'] = globals()['val_fold_'+str(i + 1) + '_with_neg'].drop(columns = ['index'])
    # print('val_fold_'+str(i + 1) + '_with_neg' + ' =\n',globals()['val_fold_'+str(i + 1) + '_with_neg'])

### Test Data

In [ ]:
for i in range(name):
    globals()['test_fold_'+str(i + 1)] = globals()['fold_'+str(i + 1)].copy()
    globals()['test_fold_'+str(i + 1)]['type'] = 'pos'
    globals()['test_fold_'+str(i + 1)] = globals()['test_fold_'+str(i + 1)].reset_index()[['uid', 'sid', 'type']]
    #print('test_fold_'+str(i + 1) + ' =\n',globals()['test_fold_'+str(i + 1)])
    globals()['test_fold_'+str(i + 1) + '_with_neg'] = pd.concat([globals()['test_fold_'+str(i+ 1)], neg_sample_df])
    globals()['test_fold_'+str(i + 1) + '_with_neg'] = globals()['test_fold_'+str(i + 1) + '_with_neg'].reset_index()[['uid', 'sid', 'type']]
    #print('test_fold_'+str(i + 1) + '_with_neg' + ' =\n',globals()['test_fold_'+str(i + 1) + '_with_neg'])

In [ ]:
# Execute only once!
test_fold_5.loc[-1] = [train_fold_5.iloc[0].values[0],train_fold_5.iloc[0].values[1],'pos']
test_fold_5.index = test_fold_5.index + 1
test_fold_5.sort_index(inplace = True)
test_fold_5_with_neg.loc[-1] = [train_fold_5.iloc[0].values[0],train_fold_5.iloc[0].values[1],'pos']
test_fold_5_with_neg.index = test_fold_5_with_neg.index + 1
test_fold_5_with_neg.sort_index(inplace = True)
train_fold_5 = train_fold_5.drop(index = 0).reset_index().drop(columns = 'index')
# print(train_fold_5)
# print(test_fold_5)
# print(test_fold_5_with_neg)

### Summarize

In [ ]:
# print('all_data =\n',all_data)
# print('neg_sample_df =\n',neg_sample_df)
# for i in range(name):
#     print('train_fold_'+str(i + 1) + ' =\n',globals()['train_fold_'+str(i + 1)])
#     print('val_fold_'+str(i + 1) + ' =\n',globals()['val_fold_'+str(i + 1)])
#     print('val_fold_'+str(i + 1) + '_with_neg' + ' =\n',globals()['val_fold_'+str(i + 1) + '_with_neg'])
#     print('test_fold_'+str(i + 1) + ' =\n',globals()['test_fold_'+str(i + 1)])
#     print('test_fold_'+str(i + 1) + '_with_neg' + ' =\n',globals()['test_fold_'+str(i + 1) + '_with_neg'])

In [ ]:

print(all_data.shape[0] == train_fold_1.shape[0] + val_fold_1.shape[0] + test_fold_1.shape[0])
print(all_data.shape[0] == train_fold_2.shape[0] + val_fold_2.shape[0] + test_fold_2.shape[0])
print(all_data.shape[0] == train_fold_3.shape[0] + val_fold_3.shape[0] + test_fold_3.shape[0])
print(all_data.shape[0] == train_fold_4.shape[0] + val_fold_4.shape[0] + test_fold_4.shape[0])
print(all_data.shape[0] == train_fold_5.shape[0] + val_fold_5.shape[0] + test_fold_5.shape[0])

In [ ]:
val_fold_1_with_neg.shape[0] == val_fold_1.shape[0] + neg_sample_df.shape[0]

In [ ]:
test_fold_1_with_neg.shape[0] == test_fold_1.shape[0] + neg_sample_df.shape[0]

### To CSV

In [ ]:
all_data.to_csv('./processed_csv_folds/all_data.csv', index = False)
neg_sample_df.to_csv('./processed_csv_folds/neg_sample.csv', index = False)

In [ ]:
for i in range(name):
    globals()['train_fold_'+str(i+1)].to_csv('./processed_csv_folds/train_fold_'+str(i+1)+'.csv', index = False)
    globals()['val_fold_'+str(i+1)].to_csv('./processed_csv_folds/val_fold_'+str(i+1)+'.csv', index = False)
    globals()['val_fold_'+str(i+1)+'_with_neg'].to_csv('./processed_csv_folds/val_fold_'+str(i+1)+'_with_neg'+'.csv', index = False)
    globals()['test_fold_'+str(i+1)].to_csv('./processed_csv_folds/test_fold_'+str(i+1)+'.csv', index = False)
    globals()['test_fold_'+str(i+1)+'_with_neg'].to_csv('./processed_csv_folds/test_fold_'+str(i+1)+'_with_neg'+'.csv', index = False)

In [ ]:
uid_pop_total = all_data.uid.value_counts().reset_index()
uid_pop_total.columns = ['uid', 'total_counts']

sid_pop_total = all_data.sid.value_counts().reset_index()
sid_pop_total.columns = ['sid', 'total_counts']

In [ ]:
for i in range(name):
    uid_pop_train = globals()['train_fold_'+str(i+1)].uid.value_counts().reset_index()
    uid_pop_train.columns = ['uid', 'train_counts']

    sid_pop_train = globals()['train_fold_'+str(i+1)].sid.value_counts().reset_index()
    sid_pop_train.columns = ['sid', 'train_counts']

In [ ]:
uid_pop_total.to_csv('./processed_csv_folds/pop_uid_all_data.csv', index = False)
sid_pop_total.to_csv('./processed_csv_folds/pop_sid_all_data.csv', index = False)

for i in range(name):
    uid_pop_train.to_csv('./processed_csv_folds/pop_uid_train_fold_'+str(i+1)+'.csv', index = False)
    sid_pop_train.to_csv('./processed_csv_folds/pop_sid_train_fold_'+str(i+1)+'.csv', index = False)

# K-Fold Epoch Negative Sample Generator

In [ ]:
user_num = all_data['uid'].max() + 1
item_num = all_data['sid'].max() + 1

In [ ]:
neg_samples_data = neg_sample_df[['uid','sid']].copy()
neg_samples_data.columns = ['user', 'item']
neg_samples_data = neg_samples_data.values.tolist()

In [ ]:
for i in range(name):
    globals()['train_data_'+str(i+1)] = globals()['train_fold_'+str(i+1)].values.tolist()
    globals()['train_mat_'+str(i+1)] = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    for x in globals()['train_data_'+str(i+1)]: globals()['train_mat_'+str(i+1)][x[0], x[1]] = 1.0

    globals()['test_data_'+str(i+1)] = globals()['test_fold_'+str(i+1)][['uid','sid']].copy()
    globals()['test_data_'+str(i+1)].columns = ['user', 'item']
    globals()['test_data_'+str(i+1)] = globals()['test_data_'+str(i+1)].values.tolist()

    globals()['val_data_'+str(i+1)] = globals()['val_fold_'+str(i+1)][['uid','sid']].copy()
    globals()['val_data_'+str(i+1)].columns = ['user', 'item']
    globals()['val_data_'+str(i+1)] = globals()['val_data_'+str(i+1)].values.tolist()

    globals()['all_mat_'+str(i+1)] = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    for x in globals()['train_data_'+str(i+1)]: globals()['all_mat_'+str(i+1)][x[0], x[1]] = 1.0
    for x in globals()['test_data_'+str(i+1)]:  globals()['all_mat_'+str(i+1)][x[0], x[1]] = 1.0
    for x in globals()['val_data_'+str(i+1)]:   globals()['all_mat_'+str(i+1)][x[0], x[1]] = 1.0
    for x in neg_samples_data:                  globals()['all_mat_'+str(i+1)][x[0], x[1]] = 1.0

In [ ]:
class BPRData(data.Dataset):
	def __init__(self, features,num_item, train_mat=None, total_mat=None, num_ng=0, is_training=None, sample_mode = None):
		super(BPRData, self).__init__()
		""" Note that the labels are only useful when training, we thus
			add them in the ng_sample() function.
		"""
		self.features = features
		self.features2 = None
		self.num_item = num_item
		self.train_mat = train_mat
		self.total_mat = total_mat
		self.num_ng = num_ng
		self.is_training = is_training
		self.sample_mode = sample_mode
		# self.labels = [0 for _ in range(len(features))]

	def ng_sample(self):
		### sample 2 pos, 2 neg

		if True:
			assert self.is_training, 'no need to sampling when testing'
			self.features_fill = []

			### add neg
			### random sample until neg is not from total_mat
			### note total_mat includes train, val, test, (test neg_samples)
			for x in self.features:
				u, pos = x[0], x[1]
				for t in range(self.num_ng):
					neg = np.random.randint(self.num_item, size = 1) ; neg = neg[0]
					while (u, neg) in self.total_mat:
						neg = np.random.randint(self.num_item, size = 1) ; neg = neg[0]
					self.features_fill.append([u, pos, neg])


	def __len__(self):
		return self.num_ng * len(self.features) if self.is_training else len(self.features)

	def __getitem__(self, idx):
		features = self.features_fill if self.is_training else self.features

		if True:
			user = features[idx][0]
			pos = features[idx][1]
			neg = features[idx][2]
			return user, pos, neg

In [ ]:
for i in range(name): globals()['train_dataset_'+str(i+1)] = BPRData(globals()['train_data_'+str(i+1)], \
                                                                         item_num, \
                                                                         globals()['train_mat_'+str(i+1)], \
                                                                         globals()['all_mat_'+str(i+1)], \
                                                                         num_ng = 1, \
                                                                         is_training = True)

#### Generate Epoch Training Data for Faster, Reproducible Training

In [ ]:
# random.seed(0)
total_epochs = 20
num_ng = 3
import pickle
for i in range(name):
    path = f'./train_samples_'+str(i+1)
    os.mkdir(path)
    print('train '+str(i+1))
    for e in range(total_epochs):
        print(e)
        train_list = []
        for j in range(num_ng):
            globals()['train_dataset_'+str(i+1)].ng_sample()
            train_samples = globals()['train_dataset_'+str(i+1)].features_fill
            train_list += train_samples
        with open(path+f'/train_samples_{e}', 'wb') as fp:
            pickle.dump(train_list, fp)